In [8]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('./src/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from package.footpaths import *
import folium
from package.logger import setup
from package import storage
import numpy as np

In [10]:
setup("INFO")

In [11]:
stops_gdf = storage.read_df('./data/cleaned/stops.csv')

In [12]:
footpaths = storage.read_any_dict("./data/footpaths.pkl")["footpaths"]

In [15]:
sample_stop = stops_gdf.iloc[0]
sample_stops = stops_gdf.sample(5)
m = folium.Map(location=[sample_stop['stop_lat'], sample_stop['stop_lon']], zoom_start=12)

for i, row in stops_gdf.iterrows():
	is_sample_stop = row.stop_id in sample_stops.stop_id.values
	radius = 3 if is_sample_stop else 1
	c = 'blue' if is_sample_stop else 'red'
	folium.CircleMarker(
		location=[row['stop_lat'], row['stop_lon']],
		radius=radius,
		color=c,
		fill=True,
		fill_color='red',
		popup=f"{row['stop_name']} ({row['stop_lat']}, {row['stop_lon']})",
	).add_to(m)


for i, row in sample_stops.iterrows():
# for i, row in stop.iterrows():
	c = "blue"
	for nearby_stop_id, walking_time_seconds in footpaths[row.stop_id].items():
		nearby_stop = stops_gdf[stops_gdf.stop_id == nearby_stop_id].iloc[0]
		folium.PolyLine(
			locations=[
				[row['stop_lat'], row['stop_lon']],
				[nearby_stop['stop_lat'], nearby_stop['stop_lon']],
			],
			color=c,
			weight=2,
			opacity=0.8,
			popup=f"{walking_time_seconds} seconds",
		).add_to(m)

		# add circle markers for both and popup the coordinates
		folium.CircleMarker(
			location=[nearby_stop['stop_lat'], nearby_stop['stop_lon']],
			radius=3,
			color=c,
			fill=True,
			fill_color='red',
			popup=f"{nearby_stop['stop_lat']}, {nearby_stop['stop_lon']} - {nearby_stop['stop_name']}",
		).add_to(m)
		folium.CircleMarker(
			location=[row['stop_lat'], row['stop_lon']],
			radius=3,
			color=c,
			fill=True,
			fill_color='red',
			popup=f"{row['stop_lat']}, {row['stop_lon']} - {row['stop_name']}",
		).add_to(m)


m